# Pigon Object Detection for AWS DeepLens

This Jupyter Notebook is based on the AWS Groundtruth Object Detection notebook: https://github.com/aws/amazon-sagemaker-examples/tree/master/ground_truth_labeling_jobs/ground_truth_object_detection_tutorial 

In [ ]:
#!pip install imgaug

import boto3
import os
import cv2
import datetime
import json
import random
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import sagemaker
from sagemaker import get_execution_role


role = get_execution_role()
sess = sagemaker.Session()

bucket = "deeplens-pigeon-detector-alexlenk"
#base_network = "resnet-50"
base_network = "vgg-16"


In [ ]:
augmented_manifest_filename_train = 'augmented-manifest-train.manifest' # Replace with the filename for your training data.
augmented_manifest_filename_validation = 'augmented-manifest-validation.manifest' # Replace with the filename for your validation data.
s3_prefix = 'pigeon-object-detection' # Replace with the S3 prefix where your data files reside.
s3_output_path = 's3://{}/output'.format(bucket) # Replace with your desired output directory.

**Pictures of Pigons are required and need to be available in the S3 bucket** 

In [ ]:
!aws s3 sync s3://$bucket/input-pics/ ./input-pics

In [ ]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
augm = iaa.SomeOf((1, 2),
            [
                sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                # search either for all edges or for directed edges,
                # blend the result with the original image using a blobby mask
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.5, 1.0)),
                    iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                ]),
                iaa.Invert(0.05, per_channel=True), # invert color channels
                iaa.Add((-5, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.LinearContrast((0.5, 2.0))
                    )
                ]),
                iaa.LinearContrast((0.5, 1.2), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.0, 1.0)),
                #sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                #sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                #sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
            random_order=True
        )

In [ ]:
def visualize_input(img_file, annotations, classes=[]):
        import random
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg

        img = mpimg.imread(img_file)
        plt.imshow(img)
        height = img.shape[0]
        width  = img.shape[1]
        colors = dict()
        num_detections = 0
        for det in annotations:
            #(klass, score, x0, y0, x1, y1) = det
            klass = det["class_id"]
            x0 = det["left"]
            y0 = det["top"]
            x1 = (det["left"] + det["width"])
            y1 = (det["top"] + det["height"])
            
            num_detections += 1
            cls_id = int(klass)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0)
            ymin = int(y0)
            xmax = int(x1)
            ymax = int(y1)
            rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id], linewidth=3.5)
            plt.gca().add_patch(rect)
            class_name = str(cls_id)
            if classes and len(classes) > cls_id:
                class_name = classes[cls_id]
            print('{}'.format(class_name))
            plt.gca().text(xmin, ymin - 2,
                            '{:s}'.format(class_name),
                            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                    fontsize=12, color='white')

        print('Number of detections: ' + str(num_detections))
        plt.show()

In [ ]:
!rm -rf aug-input-pics
!mkdir aug-input-pics
rekognition=boto3.client('rekognition')
s3 = boto3.client('s3')

paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket, Prefix="input-pics/")

object_of_interest = "Bird"

manifest_content = []

for page in pages:
    for obj in page['Contents']:
        # process items
        if(obj["Key"][-3:] == "jpg"):
            if(os.path.isfile(obj["Key"])):
                response = rekognition.detect_labels(
                    Image={
                        'S3Object': {
                            'Bucket': bucket,
                            'Name': obj["Key"]
                        }
                    },
                    MaxLabels=123,
                    MinConfidence=0.4
                )
                #print(response)
                image_in = cv2.imread(obj["Key"])
                img_width = float(image_in.shape[1])
                img_height = float(image_in.shape[0])
                
                for i in range(10):
                    scale_percent = float(random.randint(50, 100))
                    width = int(float(img_width) * scale_percent/100)
                    height = int(float(img_height) * scale_percent/100)
                    dim = (width, height)
                    print(dim)
                    image = cv2.resize(image_in, dim)
                    
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = iaa.Sometimes(0.9, augm)(image=image)
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    
                    filename = "aug-" + obj["Key"] + str(random.randint(0, 99999)).zfill(5) + ".jpg"
                    cv2.imwrite(filename, image)
                    
                    for label in response["Labels"]:
                        if label["Name"] == object_of_interest:
                            annotations = []
                            for instance in label["Instances"]:
                                box = instance["BoundingBox"]
                                #print(box)
                                annotations.append({"class_id": 0, "left": int(box["Left"]*width), "top": int(box["Top"]*height),
                                                                        "width": int(box["Width"]*width), "height": int(box["Height"]*height)})
                            manifest_newline = { 
                                                    "source-ref": "s3://{}/{}".format(bucket, filename),
                                                    "bounding-box":
                                                    {
                                                        "image_size": [{ "width": width, "height": height, "depth":3}],
                                                        "annotations": annotations
                                                    },
                                                    "bounding-box-metadata":
                                                    {
                                                        "objects":
                                                        [
                                                            {"confidence": instance["Confidence"]/100}
                                                        ],
                                                        "class-map":
                                                        {
                                                            "0": label["Name"]
                                                        },
                                                        "type": "groundtruth/object-detection",
                                                        "human-annotated": "no",
                                                        "creation-date": datetime.datetime.today().isoformat(),
                                                        "job-name": "automated-lambda-rekognition-creation-new"
                                                    }
                                                }
                                
                            if random.randint(1,10) > 8:
                                visualize_input(filename, manifest_newline["bounding-box"]["annotations"], classes=[label["Name"]])

                            #manifest_content = manifest_content + json.dumps(manifest_newline)
                            manifest_content.append(json.dumps(manifest_newline))

    print(manifest_content)
    !aws s3 rm s3://$bucket/aug-input-pics
    !aws s3 cp ./aug-input-pics s3://$bucket/aug-input-pics --recursive

In [ ]:
s3_train_data_path = 's3://{}/{}/{}'.format(bucket, s3_prefix, augmented_manifest_filename_train)
s3_validation_data_path = 's3://{}/{}/{}'.format(bucket, s3_prefix, augmented_manifest_filename_validation)

print("Augmented manifest for training data: {}".format(s3_train_data_path))
print("Augmented manifest for validation data: {}".format(s3_validation_data_path))

In [ ]:
random.shuffle(manifest_content)



print('Preview of Augmented Manifest File Contents')
print('-------------------------------------------')
print('\n')

for i in range(2):
    print('Line {}'.format(i+1))
    print(manifest_content[i])
    print('\n')


train_split = 0.8
num_train = int(train_split * len(manifest_content))
num_val = int(len(manifest_content) - num_train)

print("Training samples: {}".format(num_train))
print("Validation samples: {}".format(num_val))




s3_client = boto3.client('s3')   
s3_client.put_object(ACL='private',
                         #Body='\n'.join(augmented_manifest_lines_cropped[0:num_train-1]),
                         Body='\n'.join(manifest_content[0:num_train]),
                         Bucket=bucket,
                         Key=s3_train_data_path.split(bucket)[1][1:])

s3_client.put_object(ACL='private',
                         #Body='\n'.join(augmented_manifest_lines_cropped[num_train:]),
                         Body='\n'.join(manifest_content[num_train+1:]),
                         Bucket=bucket,
                         Key=s3_validation_data_path.split(bucket)[1][1:])

In [ ]:
attribute_names = ["source-ref","bounding-box"] # Replace as appropriate for your augmented manifest.

In [ ]:
try:
    if attribute_names == ["source-ref","XXXX"]:
        raise Exception("The 'attribute_names' variable is set to default values. Please check your augmented manifest file for the label attribute name and set the 'attribute_names' variable accordingly.")
except NameError:
    raise Exception("The attribute_names variable is not defined. Please check your augmented manifest file for the label attribute name and set the 'attribute_names' variable accordingly.")

# Create unique job name 
job_name_prefix = 'groundtruth-augmented-manifest-demo'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp
num_training_samples = num_train
training_image = sagemaker.image_uris.retrieve('object-detection', boto3.Session().region_name, version='latest')
print(training_image)

training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": training_image, # NB. This is one of the named constants defined in the first cell.
        "TrainingInputMode": "Pipe"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": s3_output_path
    },
    "ResourceConfig": {
        "InstanceCount": 1,   
        "InstanceType": "ml.p3.2xlarge",
        "VolumeSizeInGB": 50
    },
    "TrainingJobName": job_name,
    "HyperParameters": { # NB. These hyperparameters are at the user's discretion and are beyond the scope of this demo.
         "base_network": base_network,
         "use_pretrained_model": "1",
         "num_classes": "1",
         "mini_batch_size": "8",
         "epochs": "150",
         "learning_rate": "0.01",
         "lr_scheduler_step": "120,180",
         "lr_scheduler_factor": "0.1",
         "optimizer": "rmsprop",
         "momentum": "0.9",
         "weight_decay": "0.0005",
         "overlap_threshold": "0.5",
         "nms_threshold": "0.45",
         "image_shape": "512",
         "label_width": "350",
         "num_training_samples": str(num_training_samples)
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 86400
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "AugmentedManifestFile", # NB. Augmented Manifest
                    "S3Uri": s3_train_data_path,
                    "S3DataDistributionType": "FullyReplicated",
                    "AttributeNames": attribute_names # NB. This must correspond to the JSON field names in your augmented manifest.
                }
            },
            "ContentType": "application/x-recordio",
            "RecordWrapperType": "RecordIO",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "AugmentedManifestFile", # NB. Augmented Manifest
                    "S3Uri": s3_validation_data_path,
                    "S3DataDistributionType": "FullyReplicated",
                    "AttributeNames": attribute_names # NB. This must correspond to the JSON field names in your augmented manifest.
                }
            },
            "ContentType": "application/x-recordio",
            "RecordWrapperType": "RecordIO",
            "CompressionType": "None"
        }
    ]
}
 
print('Training job name: {}'.format(job_name))
print('\nInput Data Location: {}'.format(training_params['InputDataConfig'][0]['DataSource']['S3DataSource']))

In [ ]:
client = boto3.client(service_name='sagemaker')
client.create_training_job(**training_params)

# Confirm that the training job has started
status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print('Training job current status: {}'.format(status))

In [ ]:
TrainingJobStatus = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
SecondaryStatus = client.describe_training_job(TrainingJobName=job_name)['SecondaryStatus']
print(TrainingJobStatus, SecondaryStatus)
while TrainingJobStatus !='Completed' and TrainingJobStatus!='Failed':
    time.sleep(60)
    TrainingJobStatus = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    SecondaryStatus = client.describe_training_job(TrainingJobName=job_name)['SecondaryStatus']
    print(TrainingJobStatus, SecondaryStatus)

In [ ]:
%time
import boto3
from time import gmtime, strftime
from sagemaker.amazon.amazon_estimator import get_image_uri

sage = boto3.Session().client(service_name='sagemaker') 

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name="golfball-object-detection-model" + timestamp
print(model_name)
info = sage.describe_training_job(TrainingJobName=job_name)
model_src = info['ModelArtifacts']['S3ModelArtifacts']
model_data = model_src
print(model_data)

hosting_image = get_image_uri(boto3.Session().region_name, 'object-detection')

primary_container = {
    'Image': hosting_image,
    'ModelDataUrl': model_data,
}

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

sage = boto3.Session().client(service_name='sagemaker') 
info = sage.describe_training_job(TrainingJobName=job_name)
#timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = job_name_prefix + '-epc' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.t2.medium',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

%time
import time
sagemaker = boto3.client('sagemaker')

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = job_name_prefix + '-ep' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sagemaker.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))


EndpointStatus = client.describe_endpoint( EndpointName=endpoint_name )["EndpointStatus"]
while EndpointStatus !='InService' and EndpointStatus!='Failed':
    time.sleep(10)
    EndpointStatus = client.describe_endpoint( EndpointName=endpoint_name )["EndpointStatus"]

In [ ]:
%matplotlib inline
def visualize_detection(img, dets, classes=[], thresh=0.6):
        """
        visualize detections in one image
        Parameters:
        ----------
        img : numpy.array
            image, in bgr format
        dets : numpy.array
            ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
            each row is one object
        classes : tuple or list of str
            class names
        thresh : float
            score threshold
        """
        import random
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg

        #img = mpimg.imread(img_file)
        plt.imshow(img)
        height = img.shape[0]
        width  = img.shape[1]
        colors = dict()
        num_detections = 0
        for det in dets:
            (klass, score, x0, y0, x1, y1) = det
            if score < thresh:
                continue
            num_detections += 1
            cls_id = int(klass)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0 * width)
            ymin = int(y0 * height)
            xmax = int(x1 * width)
            ymax = int(y1 * height)
            rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id], linewidth=3.5)
            plt.gca().add_patch(rect)
            class_name = str(cls_id)
            if classes and len(classes) > cls_id:
                class_name = classes[cls_id]
            print('{},{}'.format(class_name,score))
            plt.gca().text(xmin, ymin - 2,
                            '{:s} {:.3f}'.format(class_name, score),
                            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                    fontsize=12, color='white')

        print('Number of detections: ' + str(num_detections))
        plt.show()

In [ ]:
import cv2

runtime = boto3.client(service_name='runtime.sagemaker')
import matplotlib.pyplot as plt
%matplotlib inline

#image_shape = 300
#filename = "resize-frame.jpg"
filename = "maxresdefault.jpg"
#filename = "frame200.jpeg"
image = cv2.imread(filename)
#image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#x_start = 700
#x_stop = 1600
#y_start = 200
#y_stop = 1000

#image = image[y_start:y_stop, x_start:x_stop]
scale_percent = 50
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
#width = int((x_stop-x_start) * scale_percent / 100)
#height = int((y_stop-y_start) * scale_percent / 100)
dim = (width, height)


print(dim)

image = cv2.resize(image,dim)
cv2.imwrite("resized-" + filename, image)

#b = ''
#with open("resized-" + filename, 'rb') as image:
#    f = image.read()
#    b = bytearray(f)
    
b = bytearray(cv2.imencode('.jpg', image)[1].tostring())

endpoint_response = runtime.invoke_endpoint(
                                 EndpointName=endpoint_name,
                                 ContentType='image/jpeg',
                                 Body=b)
results    = endpoint_response['Body'].read()
detections = json.loads(results)
#print(results)

visualize_detection(image,
                    detections['prediction'],
                    ["Pigeon"], 0.15)

In [ ]:
image_shape = 512
#!pip install 'mxnet==1.6.0'

#!rm -rf incubator-mxnet
#!git clone https://github.com/apache/incubator-mxnet --branch "v1.8.x"

sage = boto3.Session().client(service_name='sagemaker') 
info = sage.describe_training_job(TrainingJobName=job_name)
model_src = info['ModelArtifacts']['S3ModelArtifacts']
print(model_src)

network_id = base_network.replace("-", "")
network_id = "resnet50"
if network_id == "vgg16":
    network_id = "vgg16_reduced"
model_patched = model_src.replace("model.tar.gz", "patched_model_" + network_id + "_" + str(image_shape) + ".tar.gz")
model_patched = model_patched.replace("s3://", "s3://deeplens-")

!rm -rf ./tmp && mkdir ./tmp
!aws s3 cp $model_src ./tmp/
!cd ./tmp && tar xvfz model.tar.gz
!gunzip -c ./tmp/model.tar.gz | tar -C ./tmp -xopf -
network_name = "ssd_" + str(network_id) + "_" + str(image_shape)
print(network_name)
!mv ./tmp/*-0000.params ./tmp/$network_name-0000.params
!mv ./tmp/*-symbol.json ./tmp/$network_name-symbol.json

!python ./incubator-mxnet/example/ssd/deploy.py --network $network_id --data-shape $image_shape --num-class 1 --prefix tmp/ssd_

!rm ./tmp/ssd_* && rm ./tmp/model.tar.gz
!tar -cvzf ./patched_model.tar.gz -C tmp ./deploy_$network_name-0000.params ./deploy_$network_name-symbol.json ./hyperparams.json
#!aws s3 cp patched_model.tar.gz $model_patched
out_name = "s3://{}/patched_model_{}_{}.tar.gz".format(bucket, network_id, image_shape)
!aws s3 cp patched_model.tar.gz $out_name


In [ ]:
cloud_output = []
cloud_output.append({"test": "test"})

In [ ]:
print(cloud_output)